In [25]:
# ETL/EDA of Enam Biswas's IMDb Largest Review Dataset: https://www.kaggle.com/ebiswas/imdb-review-dataset
# Notebook by Frank Feder
# Goals: 
#   Clean and explore the data to get high-level summaries and identify points of interest for later analyses
#   Keep kernel novice-friendly by explaining busy lines, printing results in plain english
#   Automate the creation of a ready-to-analyze dataframe of relevant data from all six json sourcefiles

In [26]:
# OUTSTANDING TODO
# QUESTIONS
#   Is there a correlation between average review rating and date of review?
#       Do certain weekdays/months/holidays correlate to changes in average ratings?
#   A minority of movies get most of the review activity, could we use a model to a predict if a movie to be in this minority?
#       This might be more interesting after linking other movie data to use as predictors
#   What are IMDB reviewers' "highest ranked" movies by average rating?
#   Can we detect overlap between fandoms through reviewers who rate multiple titles?
#   Who are the most prolific reviewers on IMDB? What % of all reviews are posted by these reviewers?
#   What distinguishes a helpful review from one that gets voted unhelpful?
#       Are there "controversial" reviews that have high amounts of votes both ways?
#       What distinguishes reviews that generate engagement from those that receive few votes either way? 

# WRANGLING
#   Change pipeline to not drop ReviewerID and Helpful at start, to allow analysis on those columns
#       Break Helpful column into helpful_votes and unhelpful_votes
#   Connect this data with movie data from other sources, to link genre/director/actors/release date info
#       This data does not provide the IMBD ID (format "tt1234567") can we effectively match by Title/Year?
#   Create separate dataframe for analyzing reviews of ongoing/finished shows
#       How does avg. rating change over a season for a given show?
#       Can we see public opinion turn against Game of Thrones as its last season aired?
#   Create separate dataframe for analyzing reviews of of "TV Movie" or "Video" entries

In [27]:
# Import dependencies
import json # reading in source file
import pandas as pd
import re # regular expressions
import time

In [28]:
## Load data into dataframe
#dataDir = "ReviewData" # Replace with your data dir
#files_to_load = ["part-01.json", "part-02.json", "part-03.json", "part-04.json", "part-05.json", "part-06.json"]
#reviews_json = list()
#
#startTime = time.time()
#for current_file in files_to_load:
#    print(f"Starting load of {current_file}...")
#    with open(f"{dataDir}/{current_file}", mode='r') as file:
#        new_reviews = json.load(file)
#        for review in new_reviews:
#            reviews_json.append(review)
#    print(f"Finished load of {current_file} at {time.time() - startTime:.2f} total seconds elapsed")
#print(f"Loading complete after {time.time() - startTime:.2f} seconds, {len(reviews_json):,} items in reviews_json")
#
#reviews_total_count = len(reviews_json)
#print(f"Total reviews in the working data: {reviews_total_count:,}")
#
## Convert list of dicts to pandas dataframe
#reviews_df_raw = pd.DataFrame(reviews_json)

In [29]:
# TESTING (OR LIMITED RESOURCES) MODE: COMMENT OUT ABOVE BLOCK AND UNCOMMENT BELOW BEFORE RUNNING NOTEBOOK
part_one_filepath = 'ReviewData/part-01.json' # Replace with your filepath
with open(part_one_filepath, mode='r') as file:
    reviews_json = json.load(file)

reviews_total_count = len(reviews_json)
print(f"Total reviews in the working data: {reviews_total_count:,}")

# Remove the [...] piece below to test with the full ~1mil records in part-01.json
reviews_df_raw = pd.DataFrame(reviews_json[0:100000]) # Use 100,000 records, about 10% of first chunk


Total reviews in the working data: 1,010,293


In [30]:
# Peek at the data before we get started
reviews_df_raw.head(10)

,review_id,reviewer,movie,rating,review_summary,review_date,spoiler_tag,review_detail,helpful
0,rw5704482,raeldor-96879,After Life (2019– ),9,Very Strong Season 2,3 May 2020,0,"I enjoyed the first season, but I must say I t...","[1, 1]"
1,rw5704483,dosleeb,The Valhalla Murders (2019– ),6,Icelandic detectives?,3 May 2020,0,I know Iceland is a small country and police d...,"[2, 2]"
2,rw5704484,brightconscious,Special OPS (2020– ),7,Nothing special,3 May 2020,0,"Except K K , no other actor looks comfortable ...","[0, 0]"
3,rw5704485,gasconyway,#BlackAF (2020– ),8,Good but,3 May 2020,0,I'm guessing that as a 62 year old white woman...,"[5, 9]"
4,rw5704487,mmason-15867,The Droving (2020),2,An honest review,3 May 2020,0,Here's the truth. There's not much to this mov...,"[26, 41]"
5,rw5704488,schroederagustavo,All About Eve (1950),10,Amazing,3 May 2020,0,Having seen this film for the first time today...,"[0, 1]"
6,rw5704489,welhof1,Runaway Train (1985),7,Impressive action scenes!,3 May 2020,0,The movie had some very impressive scenes. Esp...,"[0, 1]"
7,rw5704490,Evastar,Iron Fist (2017–2018),9,Another great Netflix Marvel show!,3 May 2020,0,I loved it from the first episode! And I could...,"[7, 9]"
8,rw5704491,tioeta,The Half of It (I) (2020),4,Needed the other half of the movie to cover up...,3 May 2020,0,I see that Netflix has a teenage/kids audience...,"[16, 26]"
9,rw5704492,stephenrifkin,This Is Us (2016– ),2,All the Pearsons are high maintenance and self...,3 May 2020,0,This is the show for people for whom nothing e...,"[1, 5]"


In [31]:
# Remove uninteresting columns for efficiency
reviews_df_raw.drop(["review_id", "reviewer", "helpful"], axis=1, inplace=True)

In [32]:
# Check for na's
reviews_df_raw.isna().sum()

movie                0
rating            8316
review_summary       0
review_date          0
spoiler_tag          0
review_detail        0
dtype: int64

In [33]:
# Remove na's from rating, the only col with na's
raw_records_count = len(reviews_df_raw["movie"])

reviews_df_trim = reviews_df_raw[reviews_df_raw["rating"].notna()]
trim_records_count = len(reviews_df_trim["movie"])

pct_with_rating = trim_records_count / raw_records_count * 100

print(f"After removing records with \'na\' rating {trim_records_count:,} remain out of original {raw_records_count:,} ({pct_with_rating:.2f}%)") 

After removing records with 'na' rating 91,684 remain out of original 100,000 (91.68%)


In [34]:
# Confirm removing na's was successful
if reviews_df_trim.isna().sum().sum() == 0:
    print("No remaining na's in working data.")
else:
    raise SystemExit(f"\'na\' values still present: \n{reviews_df_trim.isna().sum()}")

No remaining na's in working data.


In [35]:
# Check types of remaining columns so we know what to fix
# "object" in python is ok for text (movie, review_summary, and review_detail)
# Others we can correct as part of our initial cleaning
reviews_df_trim.dtypes

movie             object
rating            object
review_summary    object
review_date       object
spoiler_tag        int64
review_detail     object
dtype: object

In [36]:
# Cast ratings as int (safe now that NaNs have been removed)
reviews_df_trim["rating"] = reviews_df_trim["rating"].astype(int)

print("ratings summary stats")
print("Avg: ", reviews_df_trim["rating"].mean())
print("Max: ", reviews_df_trim["rating"].max())
print("Min: ", reviews_df_trim["rating"].min())
print("Med: ", reviews_df_trim["rating"].median())

ratings summary stats
Avg:  6.743728458618734
Max:  10
Min:  1
Med:  7.0


In [37]:
# Cast spoiler_tag as boolean
reviews_df_trim["spoiler_tag"] = reviews_df_trim["spoiler_tag"] == 1
spoiler_value_counts = reviews_df_trim["spoiler_tag"].value_counts()
current_records_count = len(reviews_df_trim["spoiler_tag"])
spoiler_count = spoiler_value_counts[1]
spoiler_pct = spoiler_count / current_records_count * 100
print(f"Out of {current_records_count:,} reviews {spoiler_count:,} are spoilers ({spoiler_pct:.2f}%)\n")
print(spoiler_value_counts)

Out of 91,684 reviews 19,757 are spoilers (21.55%)

False    71927
True     19757
Name: spoiler_tag, dtype: int64


In [38]:
# Cast review_date as datetime
reviews_df_trim["review_date"] = pd.to_datetime(reviews_df_trim["review_date"])

print("review_date Summary Stats")
print("Avg: ", reviews_df_trim["review_date"].mean())
print("Max: ", reviews_df_trim["review_date"].max())
print("Min: ", reviews_df_trim["review_date"].min())

review_date Summary Stats
Avg:  2017-05-05 11:44:06.324331264
Max:  2020-09-05 00:00:00
Min:  2008-02-01 00:00:00


In [39]:
# Check types once more to confirm recent changes
print(reviews_df_trim.dtypes)

# Peek at data to check our progress
reviews_df_trim.head()

movie                     object
rating                     int32
review_summary            object
review_date       datetime64[ns]
spoiler_tag                 bool
review_detail             object
dtype: object


,movie,rating,review_summary,review_date,spoiler_tag,review_detail
0,After Life (2019– ),9,Very Strong Season 2,2020-05-03,False,"I enjoyed the first season, but I must say I t..."
1,The Valhalla Murders (2019– ),6,Icelandic detectives?,2020-05-03,False,I know Iceland is a small country and police d...
2,Special OPS (2020– ),7,Nothing special,2020-05-03,False,"Except K K , no other actor looks comfortable ..."
3,#BlackAF (2020– ),8,Good but,2020-05-03,False,I'm guessing that as a 62 year old white woman...
4,The Droving (2020),2,An honest review,2020-05-03,False,Here's the truth. There's not much to this mov...


In [40]:
# Separate Year using regular expressions

# Most common pattern: "Title (2000)" or "Title (2000- )" or "Title (2000-2001)"
# Also common pattern is "Title (2000) Season #, Episode #"
# There are weird ones like 'The West: The People (1500-1806) (1996) Season 1, Episode 1'
# Another: 'Red Riding: The Year of Our Lord 1980 (2009 TV Movie)'
# Another: 'The Adventures of Moby Dick (1996 Video)'

# This is also a good opportunity to remove TV shows from our working data
# We want to cut out TV shows, but TV Movies and Video should be kept
# Any dashes in the year parsing indicate it's a show, and should not be carried forward in the analysis

yearPattern = r'(?:\()(\d{4})(?!\-)?(?:\d{4})?(?:\sTV\sMovie)?(?:\sVideo)?(?:\s?\)$)(?!\sSeason\s\d+\,?\sEpisode\s\d+$)' 

In [41]:
# Use regex to pull years from movie column

yearExtract = reviews_df_trim["movie"].str.extract(yearPattern)
# Some return two years (populating index 0 and 1 of return array)
# Ideally, if all records were properly formatted movies, we'd get no na's in the [0] slot and all na's in the [1]
yearExtract.isna().sum()

0    22803
dtype: int64

In [42]:
# Check records where not even one year was pulled through regex, these should all be shows
# Need specify [0] here because regex returns a list (in this case a list of length 1) 
# Use list() on returned series to see full text of the column
list(reviews_df_trim[reviews_df_trim["movie"].str.extract(yearPattern)[0].isna()]["movie"])[0:15]

['After Life (2019– )',
 'The Valhalla Murders (2019– )',
 'Special OPS (2020– )',
 '#BlackAF (2020– )',
 'Iron Fist (2017–2018)',
 'This Is Us (2016– )',
 'Matchmaker Mysteries: A Fatal Romance (2020) Season 1, Episode 2',
 'Brooklyn Nine-Nine (2013– )',
 'After Life (2019– )',
 'The Raikar Case (2020– )',
 'Vida (2018–2020)',
 'Bloodline: Part 13 (2015) Season 1, Episode 13',
 'Jia you er nü (2005– )',
 'After Life (2019– )',
 'Four More Shots Please (2019– )']

In [43]:
# We've used regex to pull the year from every record that matches our formatting
# We can now create a dataframe excluding all TV show reviews, by only taking records that match our regex  
movie_reviews_df = reviews_df_trim[reviews_df_trim["movie"].str.extract(yearPattern)[0].notna()]

before_regex_count = len(reviews_df_trim["movie"])
after_regex_count = len(movie_reviews_df["movie"])
pct_not_movie = after_regex_count / before_regex_count * 100

print(f"After removing TV Series, {after_regex_count:,} reviews remain out of {before_regex_count:,} ({pct_not_movie:.2f}%).")

After removing TV Series, 68,881 reviews remain out of 91,684 (75.13%).


In [44]:
# Create year column to hold parsed year info
movie_reviews_df["year"] = movie_reviews_df["movie"].str.extract(yearPattern)[0]

In [45]:
# Pull name from movies column
namePattern = r'^(.+)(?:\s\()'

In [46]:
# This pattern is simpler, so we'll just quickly check the regex is matching correctly
nameExtract = movie_reviews_df["movie"].str.extract(namePattern)
nameExtract[0:10]

,0
4,The Droving
5,All About Eve
6,Runaway Train
8,The Half of It (I)
10,Closure (I)
11,Unstoppable
13,Beastie Boys Story
14,"Ruben Brandt, Collector"
15,Some Kind of Hate
16,Cube Zero


In [47]:
# Create column to hold parsed Title info
movie_reviews_df["title"] = movie_reviews_df["movie"].str.extract(namePattern)

movie_reviews_df.head()

,movie,rating,review_summary,review_date,spoiler_tag,review_detail,year,title
4,The Droving (2020),2,An honest review,2020-05-03,False,Here's the truth. There's not much to this mov...,2020,The Droving
5,All About Eve (1950),10,Amazing,2020-05-03,False,Having seen this film for the first time today...,1950,All About Eve
6,Runaway Train (1985),7,Impressive action scenes!,2020-05-03,False,The movie had some very impressive scenes. Esp...,1985,Runaway Train
8,The Half of It (I) (2020),4,Needed the other half of the movie to cover up...,2020-05-03,False,I see that Netflix has a teenage/kids audience...,2020,The Half of It (I)
10,Closure (I) (2018),9,Fun and intriguing,2020-05-03,False,This is a fun and intriguing mystery. The acti...,2018,Closure (I)


In [48]:
bad_titles_count = len(movie_reviews_df[movie_reviews_df["title"].isna()])
before_titles_cleaning_count = len(movie_reviews_df["title"])
if (bad_titles_count > 0):
    print(f"Bad titles detected: {bad_titles_count:,}")
    print(f"Before dropping title na's: {before_titles_cleaning_count:,}")
    print("Dropping records with irregularly formatted names: ")
    print(movie_reviews_df[movie_reviews_df["title"].isna()]["movie"].value_counts()) # List to display all
    movie_reviews_df.drop(movie_reviews_df[movie_reviews_df["title"].isna()].index, inplace = True)
else:
    print("No na values detected in title column.")

after_titles_cleaning_count = len(movie_reviews_df["title"])
print(f"After cleaning steps: {after_titles_cleaning_count:,} reviews remaining out of initial {reviews_total_count:,}")
clean_reviews_df = movie_reviews_df

No na values detected in title column.
After cleaning steps: 68,881 reviews remaining out of initial 1,010,293


In [49]:
#clean_reviews_df['rating'].plot(kind='hist')
clean_reviews_df['rating'].describe()[1:]

mean     6.575732
std      2.936492
min      1.000000
25%      5.000000
50%      7.000000
75%      9.000000
max     10.000000
Name: rating, dtype: float64

In [50]:
# Which release years have the most review activity?
clean_reviews_df.groupby(['year']).count()['movie'].sort_values(ascending=False).head()

year
2020    9445
2019    9259
2008    4236
2013    4183
2015    3209
Name: movie, dtype: int64

In [51]:
# Overall stats of years with reviews in working data
clean_reviews_df['year'] = clean_reviews_df['year'].astype(str).astype(int)
clean_reviews_df['year'].describe()[1:] # "count", the first index, is excluded to keep the format nice

mean    2004.520056
std       21.168614
min     1878.000000
25%     2002.000000
50%     2013.000000
75%     2019.000000
max     2021.000000
Name: year, dtype: float64

In [52]:
# Verify oldest movie... chances are whoever is commenting on ancient film would mention how old it is
oldest_movie_review = clean_reviews_df.loc[clean_reviews_df["year"] == clean_reviews_df['year'].describe()["min"]]
print(list(oldest_movie_review["movie"])[0])# List() is an easy way to display a long string inside a series
list(oldest_movie_review["review_detail"])[0] # Use the [0] index in case there are multiple for the oldest movie

Sallie Gardner at a Gallop (1878)


"So this is it. Interestingly enough, this is also the only movie-related work by Eadweard Muybridge. I'm truly curious how people perceived and reacted to these 3 seconds back in the 1880s.Muybridge certainly made a good choice for picking this subject, a majestic horse and its rider with a strong contrast in color between his shirt and his breeches. Surely a smart move to use the animal's hooves as the trigger for making the shots. If you take a close enough look you can see the numbers of the photographs he used in the bottom left. What I also liked about it is all the different shadows and shades altering constantly. It must have been a unique experience to see something in motion that, at the point of time when they saw it, was already long gone."

In [53]:
# Verify newest movie... 2022?
newest_movie_review = clean_reviews_df.loc[clean_reviews_df["year"] == clean_reviews_df['year'].describe()["max"]]
print(list(newest_movie_review["movie"])[0])# List() is an easy way to display a long string inside a series
list(newest_movie_review["review_detail"])[0] # Use the [0] index in case there are multiple for the oldest movie

Deported (2021)


'Really Funny movie. Cool Story with a great cast. Loved it! i thoroughly enjoyed it along-with my friends.'

In [54]:
# Uncomment below to create a 4.9gb .csv file in working folder, to be used for EDA
# clean_reviews_df.to_csv("imdbReviewsClean.csv", sep="\t", index = False)
clean_reviews_df.to_csv("imdbReviewsSmall.csv", sep="\t", index = False)